# SQL Practice 11/20/2022

## Q1

Given a table of posts, for each user who posted at least twice in 2021, write a query to find the number of days between each users firts post and last post of the year. The output should include the user and the number of days between their first and last post.

Data Information

`posts` **Table**

Column Name | Type
------------|----
user_id | integer
post_id | integer
post_date | timestamp
post_content | string

post_date is stored as month-day-year hour:minute:second

My solution is below:

```SQL
SELECT user_id,
      (MAX(post_date::date) - MIN(post_date::date)) AS days_between
  FROM posts
  
  WHERE DATE_PART('year', post_date::date) = 2021
  
  GROUP BY user_id
    HAVING COUNT(*) > 1
;
```

At first I thought we might need to use a `ROW_NUMBER()` partitioning by the `user_id` to identify the number of times someone has posted, then somehow get the difference between the last post and the first post by using the highest and smallest row number. However, this was overcomplicating things. I had forgotten that you could apply `MAX` and `MIN` functions to dates. It should also be noted that the double colon notation "::" is meant to convert the timestamp data type into a date. Lastly, I didn't realize that you could use aggregate functions in the `HAVING` clause without calling it as a column in the `SELECT` statement.

This also assumes that we don't have access to the `YEAR()` or `DATEDIFF()` functions.

You could also use the `EXTRACT()` function instead of `DATE_PART()` as `EXTRACT()` is used in standard SQL, but `DATE_PART()` is specific to postreSQL. That query would be:

```SQL
SELECT user_id,
      (MAX(post_date::date) - MIN(post_date::date)) AS days_between
  FROM posts
  
  WHERE EXTRACT(YEAR FROM post_date) = 2021
  
  GROUP BY user_id
    HAVING COUNT(*) > 1
;
```

## Q2

Given the tables below, write a query to find the top 2 users who sent the most internal messages in August of 2022. Dislpay the ID's of these two users and the number of messages they sent, ordered by the number of messages in descending order.

Data Information

`messages` **Table**

Column Name | Type
------------|------
message_id | integer
sender_id | integer
reciever_id | integer
content | varchar
sent_date | datetime

`sent_date` is stored as month-day-year hour:minute:second

You can assume that no two users have the same number of messages.

My solution is below:

```SQL
SELECT sender_id,
      COUNT(message_id) AS message_count
  FROM messages
  
  WHERE EXTRACT(MONTH FROM sent_date) = 08 
      AND EXTRACT(YEAR FROM sent_date) = 2022
  
  GROUP BY sender_id
  
  ORDER BY message_count DESC
  
  LIMIT 2
;
```

We are once again needing to filter by specific date information, so we can use `EXTRACT()` to filter by month and year. Lastly, since we can assume that there are no two users with the same number of messages, we can use the `LIMIT` clause together with the `ORDER BY` clause to only show the top two.

## Q3

Given the two tables below containing information on trades and users, write a query to list the top three cities that have the most completed trade orders in descending order. Output only the city and the number of orders.

Data Information:

`trades` **Table**

Column Name | Type
------------|------
order_id | integer
user_id | integer
price | decimal
quantity | integer
status | string
timestamp | datetime

`status` can only be either `"Completed"`, or `"Canceled"`

`users` **Table**

Column Name | Type
------------|-------
user_id | integer
city | string
email | string
sign_up_date | datetime

My solution is below

```SQL
WITH joined AS (
  SELECT trades.order_id, 
      trades.status, 
      users.city
      
      FROM trades LEFT JOIN users
        ON trades.user_id = users.user_id
      )

SELECT city, COUNT(order_id) AS total_orders
  FROM joined

  WHERE status = 'Completed'
  
  GROUP BY city
  
  ORDER BY total_orders DESC
  
  LIMIT 3
;
```

You can also write this without using a `WITH` statement as follows.

```SQL
SELECT users.city, COUNT(trades.order_id) AS total_orders
  FROM trades LEFT JOIN users
    ON trades.user_id = users.user_id

  WHERE trades.status = 'Completed'
  
  GROUP BY users.city
  
  ORDER BY total_orders DESC
  
  LIMIT 3
;
```

## Q4

Given the events table on app analytics, write a query to get the app's click-through rate (CTR%) in 2022. Your output should be in the form of a percentage rounded to two decimal places.

Data Information

`events` **Table**

Column Name | Type
------------|------
app_id | integer
event_type | string
timestamp | datetime

My solution is below

```SQL
WITH added AS (
  SELECT app_id,
    COUNT(CASE WHEN event_type = 'click' THEN 1 END) AS clicks,
    COUNT(CASE WHEN event_type = 'impression' THEN 1 END) AS impressions
    
    FROM events
    
    WHERE EXTRACT(YEAR FROM timestamp) = 2022
    
    GROUP BY app_id
)
    
SELECT app_id, ROUND(100.0*clicks / impressions, 2) FROM added
;
```

I realize that my original solution does something unecessary. Instead of using a `WITH` clause to create new columns clicks and impressions, then doing the calculations on those columns in a different `SELECT` statement, I can do the calculations using the `COUNT(CASE)` statements directly.

```SQL
SELECT 
  app_id,
  ROUND(100.0 * 
    COUNT(CASE WHEN event_type = 'click' THEN 1 ELSE NULL END) / 
    COUNT(CASE WHEN event_type = 'impression' THEN 1 ELSE NULL END)
    , 2) AS ctr
    
  FROM events
    
  WHERE EXTRACT(YEAR FROM timestamp) = 2022
    
  GROUP BY app_id
    
;
```

Lastly, another important point is that we want the CTR to be a float, but the calculations are being done on integers. So we multiply by `100.0` which converts the datatype to a float.